In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# Load the first CSV file
df1 = pd.read_csv('./raw_data/Raw_Sea_Level_Data/sea_levels_2015.csv')
df1['year'] = pd.to_datetime(df1['Time']).dt.year
df1_grouped = df1.groupby('year')['GMSL'].mean().reset_index()

# Load the second CSV file
df2 = pd.read_csv('./raw_data/Raw_Sea_Level_Data/sealevel.csv')
df2_selected = df2[['year', 'GMSL_GIA']].copy()
df2_selected.rename(columns={'year': 'year', 'GMSL_GIA': 'GMSL'}, inplace=True)

# Filter df2_selected to keep only rows where the year is 1994 or later
df2_selected = df2_selected[df2_selected['year'] >= 1994]

# Merge the two DataFrames on the 'Year' column, prioritizing df1 values
df_combined = pd.merge(df1_grouped, df2_selected, on='year', how='outer', suffixes=('_df1', '_df2'))
df_combined['GMSL'] = df_combined['GMSL_df1'].combine_first(df_combined['GMSL_df2'])
df_combined = df_combined[['year', 'GMSL']]

# Average the GMSL values for each year
df_combavg = df_combined.groupby('year')['GMSL'].mean().reset_index()

# Round the GMSL column to 3 decimal places
df_combavg['GMSL'] = df_combavg['GMSL'].round(3)
# Save the final DataFrame to a CSV file
df_combavg.to_csv('./Outputs/sealeveldata.csv', index=False)

In [35]:
# Load the CSV file
df = pd.read_csv('./raw_data/population/World Population Growth.csv')

# Select the required columns
df_selected = df[['year', 'Population', 'Yearly Growth %']].copy()

# Convert the Population column to a numeric type
df_selected.loc[:, 'Population'] = df_selected['Population'].str.replace(',', '').astype(int)

# Filter the DataFrame to include only years up to 2021
df_selected = df_selected[df_selected['year'] <= 2021]

# Save the DataFrame to a new CSV file
df_selected.to_csv('./Outputs/pop_data.csv', index=False)

In [36]:
# Merge the DataFrames on the 'year' column with a left join
df_combined_data = pd.merge(df_combavg, df_selected, on='year', how='left')

# Rename the columns
df_combined_data.rename(columns={
    'GMSL': 'GMSL',
    'Population': 'population',
    'Yearly Growth %': 'popChangePercent'
}, inplace=True)

# Handle population column
df_combined_data['population'] = pd.to_numeric(df_combined_data['population'], errors='coerce')
df_combined_data['population'] = df_combined_data['population'].fillna(0)

# Handle population change percent
df_combined_data['popChangePercent'] = (
    df_combined_data['popChangePercent']
    .str.rstrip('%')
    .pipe(pd.to_numeric, errors='coerce')
    .fillna(0)
)

# Save the DataFrame to a new CSV file
df_combined_data.to_csv('./Outputs/pop_sea_data.csv', index=False)

In [40]:
#Now we need to projec what the population data was for the years 1880-1950 since there's no good data available for that I can find so this is a processing of the best backwards projections I can find

df_projected = pd.read_csv('./raw_data/Population/Projected_Historic_data/population.csv')

# Drop Code column and rename Population column
df_projected = df_projected.drop('Code', axis=1)
df_projected = df_projected.rename(columns={'Population (historical)': 'pop'})

# Define list of continents to keep
continents = ["Africa", "North America", "South America", "Oceania", "Europe", "Asia"]
# Filter DataFrame to keep only rows where Entity is in continents list
df_projected = df_projected[df_projected['Entity'].isin(continents)]
# Group by year, sum populations, and filter date range
df_yearly_total = (df_projected.groupby('Year')['pop']
                   .sum()
                   .reset_index()
                   .query('1800 <= Year <= 1950'))
df_yearly_total.to_csv('./Outputs/pop_by_continent2.csv', index=False)

# Create year-population dictionary
pop_dict = dict(zip(df_yearly_total['Year'], df_yearly_total['pop']))
# Print column names to verify

# Replace zeros with mapped population values
df_combined_data.loc[df_combined_data['population'] == 0, 'pop'] = \
    df_combined_data.loc[df_combined_data['population'] == 0, 'year'].map(pop_dict)

#Write to CSV with new filename
df_combined_data.to_csv('pop_sea_full_range.csv', index=False)

In [43]:
# List all active DataFrames and their shapes
active_dfs = [var_name for var_name in dir() if isinstance(eval(var_name), pd.DataFrame)]
# Drop all DataFrames to clear data and not confuse what's what in a long notebook
for df_name in active_dfs:
    del globals()[df_name]
print("\nRemaining DataFrames:")
[var_name for var_name in dir() if isinstance(eval(var_name), pd.DataFrame)]


Remaining DataFrames:


[]

In [47]:
df_data = pd.read_csv('./Outputs/pop_sea_full_range.csv')
# Calculate population percentage change
df_data.loc[df_data['year'].between(1881, 1950), 'popChangePercent'] = (
    (df_data['pop'] - df_data['pop'].shift(1)) / df_data['pop'].shift(1) * 100).round(3)

# Write to new CSV
df_data.to_csv('./Outputs/pop_sea_full_range_complete.csv', index=False)